Siga o tutorial para construção de um Bot no Telegram (https://core.telegram.org/bots/tutorial). Baseado no código-base deste repositório do GitHub (https://github.com/vthayashi/mikusec), monte um Google Colab integrado à API do Telegram (https://core.telegram.org/bots/samples). Grave um vídeo de até 5 minutos apresentando seu chatbot, e faça 4 perguntas diferentes com 4 respostas distintas. Desafio: implemente uma memória de curto prazo neste agente conversacional, de forma que uma pergunta realizada após outra pergunta provoque uma resposta diferente do chatbot (e.g., pergunta dependente de contexto).

# Code Setup

In [ ]:
!pip install pyTelegramBotAPI
!pip install nltk
!pip install spacy
!pip install scikit-learn

In [ ]:
import pandas as pd
from datetime import datetime
import random
import re
import telebot
import nltk
import spacy

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')
spacy.cli.download('pt_core_news_lg')


nlp = spacy.load("pt_core_news_lg")
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Train Setup


In [ ]:
# Dados para a intenção "Girar um dado"
frases_girar_dado = [
    "Gire o dado por favor",
    "Qual número saiu no dado?",
    "Dado, me diga um número",
    "Lance o dado agora",
    "Vamos ver o resultado do dado",
    "Diga-me o resultado do lançamento do dado",
    "O número que saiu no dado foi?",
    "Dado, mostre-me seu valor",
    "Girando o dado...",
    "Resultado do dado?"
]

# Dados para a intenção "Momento atual"
frases_momento_atual = [
    "Qual é o momento atual?",
    "Me informe o horário atual",
    "Hora atual, por favor",
    "Que horas são?",
    "Momento do dia?",
    "Diga-me a hora agora",
    "Qual é o relógio dizendo?",
    "Hora exata?",
    "Momento presente",
    "Hora certa?"
]

# Dados para a intenção "Nota dessa ponderada"
frases_nota_ponderada = [
    "Qual é a minha nota ponderada?",
    "Diga-me minha média ponderada",
    "Nota ponderada, por favor",
    "Quanto é minha nota final?",
    "Qual é a média ponderada?",
    "Nota dessa ponderada?",
    "Minha nota ponderada é?",
    "Calculando a nota ponderada...",
    "Resultado da ponderação das notas",
    "Nota total?"
]

# Dados para a intenção "Nota da prova"
frases_nota_prova = [
    "Qual foi a nota da prova?",
    "Nota da prova, por favor",
    "Me diga minha nota do teste",
    "Nota da avaliação?",
    "Quanto tirei na prova?",
    "Minha nota da prova é?",
    "Nota final do teste",
    "Resultado da avaliação",
    "Nota da prova foi?",
    "Qual foi meu desempenho na prova?"
]

# Criar um DataFrame com os dados
dados = {
    'intent': ['Girar um dado'] * 10 + ['Momento atual'] * 10 + ['Nota dessa ponderada'] * 10 + ['Nota da prova'] * 10,
    'phrase': frases_girar_dado + frases_momento_atual + frases_nota_ponderada + frases_nota_prova
}

df = pd.DataFrame(dados)
df

,intent,phrase
0,Girar um dado,Gire o dado por favor
1,Girar um dado,Qual número saiu no dado?
2,Girar um dado,"Dado, me diga um número"
3,Girar um dado,Lance o dado agora
4,Girar um dado,Vamos ver o resultado do dado
5,Girar um dado,Diga-me o resultado do lançamento do dado
6,Girar um dado,O número que saiu no dado foi?
7,Girar um dado,"Dado, mostre-me seu valor"
8,Girar um dado,Girando o dado...
9,Girar um dado,Resultado do dado?


# BOW pipeline

In [ ]:
def clean_text(text):
    cleaned_text = re.sub(r"[^\w\s]", "", text).lower()
    return cleaned_text

def tokenizacao(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

def remove_stopwords(array_tokens):
    tokens_filtered = [word for word in array_tokens if word.lower() not in stopwords]
    return tokens_filtered

def lemmatization(array_of_tokens):
    text = ' '.join(array_of_tokens)
    doc = nlp(text)
    tokens_lemma = [token.lemma_ for token in doc]
    return tokens_lemma

def BowVectorizer(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['processed_phrase_str'])
    df_bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    df_bow['intent'] = df['intent']
    return df_bow

def pipeline(frase):
  p1 = clean_text(frase)
  p2 = tokenizacao(p1)
  p3 = remove_stopwords(p2)
  p4 = lemmatization(p3)
  return p4

def load_and_process_data(df_data):
  df_data['processed_phrase'] = df_data['phrase'].apply(pipeline)
  df_data['processed_phrase_str'] = df_data['processed_phrase'].apply(lambda x: ' '.join(x))
  df_bow = BowVectorizer(df_data)
  return df_bow

# Prepare BOW DF

In [ ]:
df_bow = load_and_process_data(df)
df_bow

,agora,atual,avaliação,calcular,certo,dar,de,desempenho,dia,diga,...,quanto,relógio,resultado,sair,teste,tirar,total,valor,ver,intent
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Girar um dado
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,Girar um dado
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Girar um dado
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Girar um dado
4,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,Girar um dado
5,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Girar um dado
6,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,Girar um dado
7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Girar um dado
8,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Girar um dado
9,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Girar um dado


In [ ]:
def find_most_similar_intent(input_text, df_bow = df_bow):
  input_processed = pipeline(input_text)
  input_processed_str = ' '.join(input_processed)

  vectorizer = CountVectorizer(vocabulary=df_bow.columns[:-1])
  input_vectorized = vectorizer.transform([input_processed_str])

  similarities = cosine_similarity(input_vectorized, df_bow.iloc[:, :-1])

  most_similar_index = similarities.argmax()

  most_similar_intent = df_bow.iloc[most_similar_index]['intent']

  return most_similar_intent
find_most_similar_intent('Quero girar o dado')

'Girar um dado'

# BOT setup

In [ ]:
bot = telebot.TeleBot('7103530556:AAF_eX7DDvgDFbPAf9vRZS4jJC33C66bb1M')

In [ ]:
@bot.message_handler(func=lambda message: True)
def message_handler(message):
  print(f"Chegou uma mensagem:\n{message.text}")
  chat_id = message.chat.id

  intent = find_most_similar_intent(message.text)

  return_message = f"Claro! Aqui está sua resposta:\nIntenção: {intent}\n"

  if intent == "Girar um dado":
    return_message += f"Seu dado resultou em: {random.randint(1, 6)}"
  elif intent == "Momento atual":
    return_message += f"O momento atual é {datetime.now()}"
  elif intent == "Nota dessa ponderada":
    return_message += f"Essa ponderada, nem preciso dizer... Nota 10!"
  elif intent == "Nota da prova":
    return_message += f"A nota da prova do Rafa é 10!"

  bot.send_message(chat_id, return_message)

In [ ]:
bot.polling()